In [ ]:
!huggingface-cli login

In [ ]:
'''!pip install datasets
!pip install gradio
!pip install ultralytics opencv-python-headless matplotlib'''

### Chatbot Llama 3.2-1B-Instruct

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-1B-Instruct")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

## Image generation Yolo

In [ ]:
from ultralytics import YOLO
model_yolo = YOLO("yolov5s.pt")

## Text to speech Speecht5_tts

In [19]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import torch
import soundfile as sf

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model_tts = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

# Download an example of a voice embedding.
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
# Embedding for example: index 7306
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

# Carga del modelo de reconocimiento de voz (Wav2Vec2)

In [20]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
processor_asr = WhisperProcessor.from_pretrained("openai/whisper-large")
model_asr = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large")

## Function to recognize objects in an image using YOLOv5. It returns name of objects and image result path

In [22]:
import cv2
def detect_objects(image):
    
    results = model_yolo(image)

    # Extract names from detected objects
    detected_objects = [
        model_yolo.names[int(cls)] for cls in results[0].boxes.cls
    ]

    result_image_rgb = results[0].plot()
    result_image_bgr = cv2.cvtColor(result_image_rgb, cv2.COLOR_RGB2BGR)

    # Save result image
    output_path = "output_image.jpg"
    cv2.imwrite(output_path, result_image_bgr)

    return output_path, detected_objects


## Function to transform audio to text ussing Whisper

In [23]:
import librosa
def audio_to_text(audio_path):
    audio_input, _ = librosa.load(audio_path, sr=16000) 
    inputs = processor_asr(audio_input, return_tensors="pt", sampling_rate=16000)

    with torch.no_grad():
        predicted_ids = model_asr.generate(inputs.input_features)
    # Decoding logits into text
    transcription = processor_asr.batch_decode(predicted_ids, skip_special_tokens=True)[0]

    return transcription

# Function that generates text + audio response

In [25]:
def generar_respuesta(input_text, image=None, audio=None):

    if audio is not None:
        input_text = audio_to_text(audio)

    if not input_text:
        return "No se detectó texto en el audio.", None, None

    detected_objects = []
    processed_image_path = None
    # Process image if available
    if image is not None:
        processed_image_path, detected_objects = detect_objects(image)
        # Add objects detected to the imput
        if detected_objects:
            input_text += f" He detectado los siguientes objetos en la imagen: {', '.join(detected_objects)}."

    # Text generation
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(
        inputs["input_ids"],
        max_length=100,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
    )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Generating audio from text
    inputs_tts = processor(text=generated_text, return_tensors="pt")
    with torch.no_grad():
        speech = model_tts.generate_speech(
            inputs_tts["input_ids"],
            speaker_embeddings,
            vocoder=vocoder
        )
    # Save audio generated
    audio_path = "speech.wav"
    sf.write(audio_path, speech.numpy(), samplerate=16000)
    
    return generated_text, audio_path, processed_image_path


# Interfaz Gradio

In [ ]:
import gradio as gr
iface = gr.Interface(
    fn=generar_respuesta,
    inputs=[
        gr.Textbox(lines=2, label="Ingresa tu texto (o deja vacío si subes un audio)"),
        gr.Image(type="numpy", label="Sube tu imagen"),
        gr.Audio(type="filepath", label="Sube tu audio")
    ],
    outputs=[
        gr.Textbox(label="Texto generado"),
        gr.Audio(label="Audio generado"),
        gr.Image(type="filepath", label="Imagen procesada")
    ],
    title="Chatbot LLama 3.2, TTS, YOLO, Text-to-Speech y Whisper.",
)

iface.launch()
